In [ ]:
import json
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
import ast

from quic_info2 import Quic_info, is_packet_lost
from sqlog2qlog import sqlog2qlog

In [ ]:
df = pd.read_csv(os.path.join("data", "bursts.csv"))

In [ ]:
df

In [ ]:
def flatten(to_flat):
    return [e for l in [ast.literal_eval(l) for l in to_flat] for e in l]


In [ ]:
def max_0(l):
    if len(l) == 0: return -1
    return max(l)

In [ ]:
df["max_burst"] = df.apply(lambda r: max_0(ast.literal_eval(r["burst"])), axis=1)
df["sum_burst"] = df.apply(lambda r: sum(ast.literal_eval(r["burst"])), axis=1)

In [ ]:
df.sort_values(by="max_burst", ascending=False)

In [ ]:
flatten_bursts = flatten(df['burst'].values)

In [ ]:
plt.boxplot(flatten_bursts)
plt.ylabel("Log scaled")
plt.yscale("log")
plt.title("Size of loss bursts for all connections")

In [ ]:
sorted_values = np.sort(flatten_bursts)
cdf = np.arange(len(sorted_values)) / len(sorted_values)


plt.step(sorted_values, cdf)
plt.title("Size of loss bursts for all connections")
plt.xlabel("Bursts size")
plt.ylabel("Cumulative Probability")
# plt.xscale("log")

plt.xlim(xmin=0.1, xmax=50)
plt.grid()
plt.savefig("output/burst_losses_cdf.pdf")

In [ ]:
plt.hist([e for e in sorted_values if e < 50], bins=80)
# plt.xscale("log")
plt.xlim(xmin=0.1, xmax=50)
plt.ylabel("Distribution")
plt.xlabel("Burst size")
plt.savefig("output/burst_losses_pdf.pdf")

In [ ]:
isp_count = [(isp, len(df.loc[df["isp"] == isp])) for isp in df["isp"].unique()]


data = []
names = []

for isp, count in sorted(isp_count, key=lambda x:x[1], reverse=True):
    flatted = flatten(df.loc[df["isp"] == isp]["burst"].values)
    if len(flatted) > 20:
        print(isp, count)
        data.append(flatted)
        names.append(isp)
    # elif count >= 5:
    #     print("Not took : ", isp)

In [ ]:
fig = plt.figure(figsize=(8, 12))
plt.boxplot(data, vert=False)
plt.yticks(list(range(1, len(names) + 1)), names)
plt.xscale("log")
plt.xlabel("Burst size")
plt.title("Size of burst for ISP")
plt.grid()